<a href="https://colab.research.google.com/github/Prashuk2/Finetuning-FinBERT-for-Sentiment-Analysis/blob/main/minor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading data file


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving data (1).csv to data (1).csv


In [ ]:
import pandas as pd
import io
df= pd.read_csv(io.BytesIO(uploaded['data (1).csv']))

In [ ]:
df

,Content,Sentiment
0,US consumer spending dropped by a record in Ap...,Negative
1,State-run lenders require an urgent Rs 1.2 tri...,Negative
2,Apparel exporters on Wednesday urged the gover...,Negative
3,Asian shares battled to extend a global reboun...,Negative
4,After India’s sovereign credit rating fell to ...,Negative
...,...,...
29285,"The company, whose products range from insulin...",Neutral
29286,The company is yet to finalise the choice of l...,Neutral
29287,The decision to pause comes after Foxconn sent...,Neutral
29288,Given the tailwinds in the MF industry and the...,Neutral


Finetuned Finbert Model

In [ ]:

!pip install transformers datasets scikit-learn

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import f1_score, accuracy_score
import numpy as np


df['Sentiment'] = df['Sentiment'].str.lower().replace({
    'positive': 'Positive',
    'negative': 'Negative',
    'neutral': 'Neutral'
})

label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


def tokenize_function(examples):
    return tokenizer(
        examples['Content'],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

print("Tokenized training data example:", train_dataset[0])
print("Tokenized validation data example:", val_dataset[0])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


trainer.train()


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {"f1_score": f1, "accuracy": accuracy}


eval_results = trainer.evaluate()
val_predictions = trainer.predict(val_dataset)
val_metrics = compute_metrics((val_predictions.predictions, val_predictions.label_ids))


print("Validation Results:")
print(f"Loss: {eval_results['eval_loss']}")

print(f"Accuracy: {val_metrics['accuracy']}")
print(f"F1 Score: {val_metrics['f1_score']}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5858 [00:00<?, ? examples/s]

Tokenized training data example: {'Content': 'live bse live  nse live Volume Todays L/H More ×  Country\'s second-largest private sector lender HDFC Bank is expected to report more than 20 percent growth in profit in Q4FY20 and there could be some impact of likely contingent provisions on profitability.  The stock closed at Rs 910.40, up 3.33 percent, ahead of quarterly earnings scheduled to be announced on April 18. It lost more than 32 percent during the quarter ended March 2020 largely due to lockdown-led disruption in the economy after COVID-19 spread.  Brokerages largely expect net interest income during the quarter to be in the range of 12-17 percent due slow loan growth which could be around 20 percent compared to year-ago period.  "We expect loan growth to slow to around 20 percent YoY, resulting in lower NII growth at around 15 percent YoY. Retail loan growth slowdown is on account of weak volume growth in auto though the share of unsecured portfolio would continue to remain h

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.669900,0.635320
2,0.385000,0.438817
3,0.224700,0.466277


Validation Results:
Loss: 0.4662769138813019
Accuracy: 0.8764083304882212
F1 Score: 0.8761427886446944


Naive Bayes


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

df['Sentiment'] = df['Sentiment'].str.capitalize()
label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_map.keys()))


Accuracy: 0.607203823830659
Classification Report:
               precision    recall  f1-score   support

    Positive       0.64      0.69      0.66      1963
    Negative       0.59      0.68      0.63      1897
     Neutral       0.58      0.46      0.52      1998

    accuracy                           0.61      5858
   macro avg       0.61      0.61      0.60      5858
weighted avg       0.61      0.61      0.60      5858



SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

df['Sentiment'] = df['Sentiment'].str.capitalize()
label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vec, y_train)

y_pred = svm_model.predict(X_test_vec)
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_map.keys()))


SVM Accuracy: 0.7466712188460225
              precision    recall  f1-score   support

    Positive       0.76      0.79      0.78      1963
    Negative       0.77      0.79      0.78      1897
     Neutral       0.71      0.66      0.69      1998

    accuracy                           0.75      5858
   macro avg       0.75      0.75      0.75      5858
weighted avg       0.75      0.75      0.75      5858



KNN


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

df['Sentiment'] = df['Sentiment'].str.capitalize()

label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

X = df['Content']
y = df['label']

vectorizer = TfidfVectorizer(max_features=5000)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Positive', 'Negative', 'Neutral'])

print("KNN Accuracy:", accuracy)
print("Classification Report:\n", report)


KNN Accuracy: 0.6864117446227381
Classification Report:
               precision    recall  f1-score   support

    Positive       0.67      0.75      0.71      1963
    Negative       0.68      0.70      0.69      1897
     Neutral       0.71      0.61      0.66      1998

    accuracy                           0.69      5858
   macro avg       0.69      0.69      0.69      5858
weighted avg       0.69      0.69      0.69      5858



LSTM

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

label_map = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

df = df.dropna(subset=['label'])


tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Content'])

X = tokenizer.texts_to_sequences(df['Content'])
X = pad_sequences(X, maxlen=100)

y = df['label'].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


model = Sequential([
    Embedding(input_dim=5000, output_dim=64),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.1
)

loss, accuracy = model.evaluate(X_test, y_test)
print("LSTM Accuracy:", accuracy)


Epoch 1/5
659/659 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.4256 - loss: 1.0548 - val_accuracy: 0.5990 - val_loss: 0.8828
Epoch 2/5
659/659 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.6713 - loss: 0.7668 - val_accuracy: 0.6745 - val_loss: 0.7784
Epoch 3/5
659/659 ━━━━━━━━━━━━━━━━━━━━ 34s 51ms/step - accuracy: 0.7787 - loss: 0.5567 - val_accuracy: 0.7056 - val_loss: 0.7553
Epoch 4/5
659/659 ━━━━━━━━━━━━━━━━━━━━ 40s 50ms/step - accuracy: 0.8374 - loss: 0.4238 - val_accuracy: 0.7065 - val_loss: 0.7806
Epoch 5/5
659/659 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.8691 - loss: 0.3479 - val_accuracy: 0.7440 - val_loss: 0.8048
184/184 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7706 - loss: 0.6994
LSTM Accuracy: 0.7627176642417908


Finbert(pretrained)

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.nn.functional import softmax
from tqdm import tqdm


print("Original DataFrame shape:", df.shape)
df = df.dropna(subset=['Content', 'Sentiment'])
print("After dropping NaNs:", df.shape)
print("Unique Sentiment values:", df['Sentiment'].unique())
df['Sentiment'] = df['Sentiment'].str.lower().str.strip()
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
df = df[df['Sentiment'].isin(label_map.keys())]
df['label'] = df['Sentiment'].map(label_map)

_, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")


predictions = []
model.eval()

for text in tqdm(test_df['Content'], desc="Predicting"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs, dim=1).item()
    predictions.append(pred_label)


y_true = test_df['label'].tolist()
y_pred = predictions

accuracy = accuracy_score(y_true, y_pred)
print(f"\n✅ FinBERT Accuracy: {accuracy:.4f}")


Original DataFrame shape: (29290, 2)
After dropping NaNs: (29290, 2)
Unique Sentiment values: ['Negative' 'Neutral' 'Positive']


Predicting: 100%|██████████| 5858/5858 [1:17:29<00:00,  1.26it/s]


✅ FinBERT Accuracy: 0.2404
